In [1]:
import altair as alt
from pathlib import Path
import json
import pandas as pd
import numpy as np

alt.renderers.enable('altair_viewer', inline=True)

RendererRegistry.enable('altair_viewer')

In [15]:
source = pd.read_csv('2020_07_28_15_48.csv')
alt.Chart(source).mark_bar().encode(
    x='type:O',
    y='lut:Q',
    column=alt.Column('benchmark:O',
                     title="",
                     header=alt.Header(labelAngle=-20)),
    color='type:O'
)

FileNotFoundError: [Errno 2] No such file or directory: '2020_07_28_15_48.csv'

In [14]:
s

alt.Chart(...)

In [131]:
source = pd.read_csv('7_31.csv')
alt.Chart(source).mark_bar().encode(
    x='type:O',
    y='dsp:Q',
    column=alt.Column('benchmark:O',
                     title="",
                     header=alt.Header(labelAngle=-20)),
    color='type:O'
)


alt.Chart(...)

In [191]:
# compare latency diffs
a = pd.read_csv('../results/14_15_10-08_2020/data.csv')
b = pd.read_csv('../small_seq.csv')
df = pd.concat([a, b], keys=['old', 'comb']).reset_index().drop(['level_1'], axis='columns').rename({'level_0': 'type'}, axis='columns')
alt.Chart(df).mark_bar().encode(
    x='type',
    y='latency',
    column=alt.Column('benchmark',
                     title="",
                     header=alt.Header(labelAngle=-20)),
    tooltip=['latency']
)
b.loc[b['benchmark'] == 'linear-algebra-3mm', 'latency']

1    28457
Name: latency, dtype: int64

In [13]:
source = pd.read_csv('../vivado-experiments/7_31.csv')
sim = pd.read_csv('../vivado-experiments/results/2020_07_30_16_46/sim.csv')

source = source[source['type'] != 'hls_total']

for _, row in sim.iterrows():
    bench_filt = source['benchmark'] == row[0]
    type_filt = source['type'] == 'futil'
    source.loc[(bench_filt) & (type_filt), 'latency'] = row[1]

alt.Chart(source).mark_bar().encode(
    x='type:O',
    y='latency:Q',
    column=alt.Column('benchmark:O',
                     title="",
                     header=alt.Header(labelAngle=-20)),
    color='type:O',
    tooltip=['latency']
)

alt.Chart(...)

In [14]:
source = pd.read_csv('../vivado-experiments/7_31.csv')
sim = pd.read_csv('../vivado-experiments/results/2020_07_30_16_46/sim.csv')

source = source[source['type'] != 'hls_total']

for _, row in sim.iterrows():
    bench_filt = source['benchmark'] == row[0]
    type_filt = source['type'] == 'futil'
    source.loc[(bench_filt) & (type_filt), 'latency'] = row[1]
#     print(row[1])

futil = alt.Chart(source[source['type'] == 'futil']).mark_circle(size=100).encode(
    x='lut:Q',
    y='latency:Q',
    color='type:O',
    tooltip=['latency', 'lut', 'benchmark']
)

hls = alt.Chart(source[source['type'] == 'hls']).mark_circle(size=25).encode(
    x='lut:Q',
    y='latency:Q',
    color='type:O',
    tooltip=['latency', 'lut', 'benchmark']
)


base = alt.layer(futil, hls)

for bench in source['benchmark'].unique():
    df = source[source['benchmark'] == bench]
    N = 30
    lut_arr = np.linspace(df[df['type'] == 'futil']['lut'], df[df['type'] == 'hls']['lut'], num=N)
    lat_arr = np.linspace(df[df['type'] == 'futil']['latency'], df[df['type'] == 'hls']['latency'], num=N)
    df.insert(6, 'dist', [0, 11])
    for i in range(N):
        df = df.append({
           'benchmark': bench,
            'type': 'linear',
            'lut': lut_arr[i][0],
            'dsp': 0,
            'latency': lat_arr[i][0],
            'dist': N - i + 5,
            'source': 'n/a'
        }, ignore_index=True)
            
    chart = alt.Chart(df).mark_trail().encode(
        x='lut:Q',
        y='latency:Q',
        size='dist:Q',
        tooltip=['latency', 'lut']
    )
    base = alt.layer(base, chart)

base.interactive()


alt.LayerChart(...)

In [2]:
source = pd.read_csv('../vivado-experiments/7_31.csv')
sim = pd.read_csv('../vivado-experiments/results/2020_07_30_16_46/sim.csv')

# don't care about hls total for this
source = source[source['type'] != 'hls_total']

for _, row in sim.iterrows():
    bench_filt = source['benchmark'] == row[0]
    type_filt = source['type'] == 'futil'
    source.loc[(bench_filt) & (type_filt), 'latency'] = row[1]
    
for bench in source['benchmark'].unique():
    df = source[source['benchmark'] == bench]
    source = source.append({
        'benchmark': bench,
        'type': 'difference',
        'lut': df[df['type'] == 'futil']['lut'].item() - df[df['type'] == 'hls']['lut'].item(),
        'dsp': 0,
        'latency': df[df['type'] == 'futil']['latency'].item() - df[df['type'] == 'hls']['latency'].item(),
        'source': 'n/a'
    }, ignore_index=True)

src = source[source['type'] == 'difference']
# src = source

lut = alt.Chart(src).mark_bar(color='green', opacity=0.5).encode(
    x = 'benchmark',
    y = alt.Y('lut', scale=alt.Scale(type='linear', base=10)),
)
lat = alt.Chart(src).mark_bar(size=6).encode(
    x = alt.X('benchmark'),
    y = alt.Y('latency', scale=alt.Scale(type='linear', base=10))
)

(lat + lut).encode(
    tooltip = ['lut', 'latency', 'benchmark']
)

alt.LayerChart(...)

In [78]:
pd.read_csv('../results/04_11_53-08_2020/data.csv')

,benchmark,type,lut,dsp,latency,source
0,linear-algebra-cholesky,hls,863,6,1881,results/04_11_53-08_2020
1,linear-algebra-cholesky,hls_total,903,6,1881,results/04_11_53-08_2020
2,linear-algebra-cholesky,futil,2098,6,0,results/04_11_53-08_2020
3,linear-algebra-gramschmidt,hls,1252,9,12841,results/04_11_53-08_2020
4,linear-algebra-gramschmidt,hls_total,1292,9,12841,results/04_11_53-08_2020
5,linear-algebra-gramschmidt,futil,2231,9,0,results/04_11_53-08_2020


In [65]:
df = pd.read_csv('../results/14_15_10-08_2020/data.csv')
# df = pd.read_csv('../results/07_15_06-08_2020/data2.csv')
sim = pd.read_csv('../results/14_15_10-08_2020/sim.csv')
df = df[df['type'] != 'hls_total']
for _, row in sim.iterrows():
    bench_filt = df['benchmark'] == row[0]
    type_filt = df['type'] == 'futil'
    df.loc[(bench_filt) & (type_filt), 'latency'] = row[1]
df = df.sort_values(by='benchmark').reset_index(drop=True)

lut = alt.Chart(df).mark_bar().encode(
    x='type',
    y='lut',
    column=alt.Column('benchmark',
                     title="",
                     header=alt.Header(labelAngle=-20)),
    color=alt.condition('datum.type == "futil" && datum.meet_timing == 0', alt.ColorValue('red'), 'type'),
    tooltip=['latency', 'lut', 'dsp']
)

dsp = alt.Chart(df).mark_bar().encode(
    x='type',
    y='dsp:Q',
    column=alt.Column('benchmark:O',
                     title="",
                     header=alt.Header(labelAngle=-20)),
    color=alt.condition('datum.type == "futil" && datum.meet_timing == 0', alt.ColorValue('red'), 'type'),
    tooltip=['latency', 'lut', 'dsp']
)

latency = alt.Chart(df).mark_bar().encode(
    x='type',
    y='latency',
    column=alt.Column('benchmark:O',
                     title="",
                     header=alt.Header(labelAngle=-20)),
    color=alt.condition('datum.type == "futil" && datum.meet_timing == 0', alt.ColorValue('red'), 'type'),
    tooltip=['latency', 'lut', 'dsp']
)

df

,benchmark,type,lut,dsp,latency,meet_timing,source
0,linear-algebra-2mm,hls,939,12,3718.0,1,results/14_15_10-08_2020
1,linear-algebra-2mm,futil,4183,12,26358.0,1,results/14_15_10-08_2020
2,linear-algebra-3mm,hls,742,9,5043.0,1,results/14_15_10-08_2020
3,linear-algebra-3mm,futil,583,9,28457.0,1,results/14_15_10-08_2020
4,linear-algebra-atax,futil,1150,6,3601.0,1,results/14_15_10-08_2020
5,linear-algebra-atax,hls,610,6,715.0,1,results/14_15_10-08_2020
6,linear-algebra-bicg,futil,318,6,1940.0,1,results/14_15_10-08_2020
7,linear-algebra-bicg,hls,327,6,218.0,1,results/14_15_10-08_2020
8,linear-algebra-cholesky,futil,614,6,3138.0,1,results/14_15_10-08_2020
9,linear-algebra-cholesky,hls,4040,6,1385.0,1,results/14_15_10-08_2020


In [44]:
x = []
for bench in df['benchmark'].unique():
    bfilt = df['benchmark'] == bench
    hls_filt = df['type'] == 'hls'
    futil_filt = df['type'] == 'futil'
    ratio = df[(bfilt & futil_filt)]['latency'].values/df[(bfilt & hls_filt)]['latency'].values
    if not np.isnan(ratio[0]):     
        x.append(ratio[0])
np.mean(x)

5.308599012801434

In [196]:
df = pd.read_csv('../results/14_15_10-08_2020/data.csv')
# df = pd.read_csv('../results/07_15_06-08_2020/data2.csv')
sim = pd.read_csv('../results/14_15_10-08_2020/sim.csv')
seq = pd.read_csv('../small_seq.csv')
df = df[df['type'] != 'hls_total']
for _, row in sim.iterrows():
    bench_filt = df['benchmark'] == row[0]
    type_filt = df['type'] == 'futil'
    df.loc[(bench_filt) & (type_filt), 'latency'] = row[1]
df = df.sort_values(by='benchmark').reset_index(drop=True)

data = pd.DataFrame()
for bench in df['benchmark'].unique():
    rows = df[df['benchmark'] == bench]
    hls = rows[rows['type'] == 'hls']
    futil = rows[rows['type'] == 'futil']
    data = data.append({
        'benchmark': bench,
        'type': f"lut",
        'value': futil['lut'].values[0] / hls['lut'].values[0],
        'meet_timing': futil['meet_timing'].values[0],
        'equal': 1
    }, ignore_index=True)
    data = data.append({
        'benchmark': bench,
        'type': f"dsp",
        'value': futil['dsp'].values[0] / hls['dsp'].values[0],
        'meet_timing': futil['meet_timing'].values[0],
        'equal': 1
    }, ignore_index=True)
    data = data.append({
        'benchmark': bench,
        'type': f"latency",
        'value': futil['latency'].values[0] / hls['latency'].values[0],
        'meet_timing': futil['meet_timing'].values[0],
        'equal': 1
    }, ignore_index=True)
#     data = data.append({
#         'benchmark': bench,
#         'type': "latency_seq",
#         'value': seq.loc[seq['benchmark'] == bench,'latency'].values[0] / hls['latency'].values[0],
#         'meet_timing': futil['meet_timing'].values[0],
#         'equal': 1
#     }, ignore_index=True)

bars = alt.Chart().mark_bar().encode(
    x=alt.X('type', 
                axis=alt.Axis(tickOffset=1.0, offset=0.0, orient='bottom')),
    y=alt.Y('value',
            scale=alt.Scale(type='log'),
            axis=alt.Axis(grid=False)),
    
    color=alt.condition('datum.meet_timing == 0', alt.ColorValue('red'), 'type'),
)

vline = alt.Chart(data).mark_rule(color='red').encode(y='equal')

alt.layer(bars, vline, data=data).facet(
    column=alt.Column('benchmark',
                        title="",
                        header=alt.Header(labelAngle=-20),
                     ),
    spacing=0
)
# text = bars.mark_text(dy=-10).encode(
#   text='value'
# )



alt.FacetChart(...)

In [75]:
source = df
futil = alt.Chart(source[source['type'] == 'futil']).mark_circle(size=100).encode(
    x='lut:Q',
    y='latency:Q',
    color='type:O',
    tooltip=['latency', 'lut', 'benchmark']
)

hls = alt.Chart(source[source['type'] == 'hls']).mark_circle(size=25).encode(
    x='lut:Q',
    y='latency:Q',
    color='type:O',
    tooltip=['latency', 'lut', 'benchmark']
)


base = alt.layer(futil, hls)

for bench in source['benchmark'].unique():
    data = source[source['benchmark'] == bench]
    N = 30
    lut_arr = np.linspace(data[data['type'] == 'futil']['lut'], data[data['type'] == 'hls']['lut'], num=N)
    lat_arr = np.linspace(data[data['type'] == 'futil']['latency'], data[data['type'] == 'hls']['latency'], num=N)
    data.insert(6, 'dist', [0, 11])
    for i in range(N):
        data = data.append({
           'benchmark': bench,
            'type': 'linear',
            'lut': lut_arr[i][0],
            'dsp': 0,
            'latency': lat_arr[i][0],
            'dist': N - i + 5,
            'source': 'n/a'
        }, ignore_index=True)
            
    chart = alt.Chart(data).mark_trail().encode(
        x='lut:Q',
        y='latency:Q',
        size='dist:Q',
        tooltip=['latency', 'lut']
    )
    base = alt.layer(base, chart)

base.interactive()


alt.LayerChart(...)